In [1]:
import pyspark 
from pyspark.sql import SparkSession
import base64

In [2]:
from pyspark.sql.types import StringType

In [3]:
key_file = open("/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/crypto-busting-375023-6722d6967eca.json", "rb")
key = base64.b64encode(key_file.read())
key = key.decode("utf-8")

In [4]:
spark = SparkSession.builder \
    .master("yarn") \
    .appName("Spark Streaming to Pub/Sub Lite") \
    .config("spark.jars", "/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/pubsublite-spark-sql-streaming-LATEST-with-dependencies.jar") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/22 23:55:24 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/01/22 23:55:24 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/01/22 23:55:24 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/01/22 23:55:25 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [5]:
project_number = 1072423212419
location = "europe-central2"
subscription_id = "bda-coinbase-sub-lite"
df = (
    spark.readStream.format("pubsublite")
    .option(
        "pubsublite.subscription",
        f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
    )
    .option("gcp.credentials.key", key)
    .load()
)

In [6]:
# w tej kolumnie powinny być dane ale są zakodowane jako json?
df = df.withColumn('data', df.data.cast(StringType())).select('data')

In [7]:
# Print
res = df.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

23/01/22 23:56:04 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-06891958-0442-4eed-b3a5-ab605f7bc899. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/22 23:56:04 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [15]:
from pyspark.sql.functions import from_json, to_json
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, FloatType

In [9]:
JSONschema = StructType([ 
    StructField("side", StringType(), True), 
    StructField("price", StringType(), False), 
    StructField("product_id", StringType(), False),
    StructField("time", StringType(), False), 
])

In [10]:
sdf = df.withColumn("JSONData", from_json(col("data"), JSONschema)).select("JSONData.*")

In [11]:
sdf = sdf.withColumn("price", sdf.price.cast(FloatType()))

In [12]:
sdf.isStreaming

True

In [13]:
sdf.printSchema()

root
 |-- side: string (nullable = true)
 |-- price: float (nullable = true)
 |-- product_id: string (nullable = true)
 |-- time: string (nullable = true)



In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array, create_map, col, lit, when, struct
from pyspark.sql.types import BinaryType, StringType
import uuid

In [ ]:
message = (
    sdf.withColumn("key", col("time").cast(BinaryType()))
    .withColumn("data", to_json())
)